# Generative Adversarial Networks

Применение adversarial loss (более общей идеи, лежащей в основе GANов) позволило решить задачи, которые казались невозможными:

* [Машинный перевод без параллельных данных](https://arxiv.org/pdf/1710.11041.pdf)
* [Циклоганы: перевод изображений в другой домен](https://arxiv.org/abs/1703.10593)
* Колоризация и [Super Resolution](https://arxiv.org/abs/1807.02758)
* [Генерация и морфинг произвольных данных](https://arxiv.org/pdf/1809.11096.pdf) ([тут](https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/biggan_generation_with_tf_hub.ipynb#scrollTo=HuCO9tv3IKT2) можно поиграться с генерацией бургеров)
* Применения в борьбе с adversarial атаками

Вот постоянно пополняющийся список приложений GANов: https://github.com/nashory/gans-awesome-applications

Сама [статья](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf) Яна Гудфеллоу про GANы вышла в конце 2014 года и была процитирована 7687 раз за 4 года.


<img width='500px' src='https://cdn-images-1.medium.com/max/800/1*eWURQXT41pwHvDg1xDiEmw.png'>

Теперь немного формальных определений:

* Пусть $z$ — это вектор из латентного пространства, насэмпленный из нормального распределения.
* $G(z)$ обозначает функцию генератора, которая отображает латентный вектор в пространство данных. Цель $G$ — оценить истинное распределение данных $p_d$, чтобы сэмплировать данные из оцененного распределения $p_g$.
* $D(G(z))$ это вероятность (число от 0 до 1), что выход генератора $G$ является реальным изображением.

$D$ и $G$ играют в минимаксную игру, в которой $D$ старается максимизировать вероятность, что он правильно классифицирует реальные и сгенерированные сэмплы, а $G$ старается минимизировать эту вероятность:

$$\underset{G}{\text{min}} \underset{D}{\text{max}}V(D,G) = \mathbb{E}_{x\sim p_{data}(x)}\big[logD(x)\big] + \mathbb{E}_{z\sim p_{z}(z)}\big[log(1-D(G(x)))\big]$$

[Выясняется](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf), что решение в этой минимаксной игре достигается при $p_g = p_d$ (и дискриминатор в этом случае может угадывать случайно). В реальности модели не всегда могут сойтись к этой точке.

[DCGAN](https://arxiv.org/pdf/1511.06434.pdf) (Deep Convolutional GAN) называют GAN, который явно использует свёртки и транспонированные свёртки в дискриминаторе и генераторе соответственно. Откройте статью -- мы будем идти очень близко с авторами.

## Датасет
Всем надоели цифры, поэтому обучаться мы будем на датасете CelebA ([Large-scale CelebFaces Attributes](Large-scale CelebFaces Attributes)). В датасете на каждую фотку есть её аттрибуты, но мы их пока использовать не будем.

<img width='500px' src='http://mmlab.ie.cuhk.edu.hk/projects/celeba/overview.png'>

Автор, когда готовил эту тетрадку, долго думал, как загрузить датасет, чтобы всем было удобно. Это оказалось трудно, потому что прямых ссылок на него нигде нет, и, соответственно, просто сделать `!wget ...` нельзя. По удачному стечению обстоятельств, неделю назад кто-то [добавил](https://github.com/pytorch/vision/blob/master/torchvision/datasets/celeba.py) скрипты для загрузки этого датасета в сам `torchvision`, но в `pip` новая версия за такой срок ещё не успела появиться, поэтому мы обновимся напрямую из репозитория на гитхабе:

In [0]:
# модель будет обучаться долго - рекомендую сохранять промежуточные версии. В колабе это можно делать примерно так
from google.colab import drive
drive.mount('/content/drive')
#теперь для сохранения файлов вам доступна директория /content/drive

In [0]:
!pip install git+https://github.com/pytorch/vision.git

In [0]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets

In [0]:
device = torch.device('cuda:0')  # не забудьте включить GPU

image_size = 64
batch_size = 64

In [0]:
transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    # Normalize здесь приводит значения в промежуток [-1, 1]
])

dataset = datasets.CelebA('data', download=True, transform=transform)

loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [0]:
dataset[5][0]

In [0]:
# посмотрите на данные (вы писали нужный код в колоризации)
# ...

## Модель

Генератор $G$ преобразует латентный вектор $z$ в пространство данных (в нашем случае -- картинки 3x64x64). В статье используют последовательность блоков из транспонированных свёрток, BatchNorm-ов и ReLU. На выходе каждое значение лежит в [-1, 1] (мы делаем TanH), в соответствии с нормализацией, которую мы сделали раньше.

<img width='600px' src='https://pytorch.org/tutorials/_images/dcgan_generator.png'>

In [0]:
device = torch.device('cuda:0')

In [0]:
num_channels = 3
latent_size = 100
base_size = 64

G = nn.Sequential(
    # input is Z, going into a convolution
    nn.ConvTranspose2d(latent_size, base_size * 8, 4, 1, 0, bias=False),
    nn.BatchNorm2d(base_size * 8),
    nn.ReLU(True),
    
    # (base_size*8) x 4 x 4
    nn.ConvTranspose2d(base_size * 8, base_size * 4, 4, 2, 1, bias=False),
    nn.BatchNorm2d(base_size * 4),
    nn.ReLU(True),
    
    # (base_size*4) x 8 x 8
    nn.ConvTranspose2d(base_size * 4, base_size * 2, 4, 2, 1, bias=False),
    nn.BatchNorm2d(base_size * 2),
    nn.ReLU(True),
    
    # (base_size*2) x 16 x 16
    nn.ConvTranspose2d(base_size * 2, base_size, 4, 2, 1, bias=False),
    nn.BatchNorm2d(base_size),
    nn.ReLU(True),
    
    # (base_size) x 32 x 32
    nn.ConvTranspose2d(base_size, num_channels, 4, 2, 1, bias=False),
    nn.Sigmoid()
    # (num_channels) x 64 x 64
).to(device)

In [0]:
z = torch.randn(1, latent_size, 1, 1).to(device)
G(z)

Дискриминатор -- это обычный бинарный классификатор. В статье он устроен симметрично генератору: Conv2d, BatchNorm, ReLU, Conv2d... Параметры сверток можно поставить в обратную сторону.

In [0]:
D = nn.Sequential(
    # ...
    nn.Conv2d(base_size * 8, 1, 4, 2, 0, bias=False),
    nn.BatchNorm2d(1),
    nn.Sigmoid()
).to(device)

In [0]:
z = torch.randn(1, num_channels, image_size, image_size).to(device)
D(z)

В статье акцентируют внимание на необходимость нестандартной инициализации весов.

In [0]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


# apply рекурсивно применяет применяет функцию ко всем своим подмодулям
G.apply(weights_init)
D.apply(weights_init)

## Обучение

У GANов, помимо сходимости, есть проблема, что их непонятно, как сравнивать между собой, потому что у нас не один лосс, а два. Поэтому полезнее во время обучения смотреть на генерируемые картинки, а не цифры.

In [0]:
# если мы предварительно сохраняли модели и хотим запустить их, то это вот так
D.load_state_dict(torch.load('/content/drive/My Drive/D.pt')) # можно и другую директорию, но вот это прямо внутри вашего гугл диска
G.load_state_dict(torch.load('/content/drive/My Drive/G.pt'))

In [0]:
num_epochs = 5
learning_rate = 1e-3

img_list = []
G_losses = []
D_losses = []
iters = 0

optim_G = # ваш любимый оптимизатор параметров дискриминатора
optim_D = # ваш любимый оптимизатор параметров генератора
  
for epoch in range(num_epochs):
    for (data, _) in loader:
        # Обучать GANы всегда долго, и мы хотим по максимуму переиспользовать вычисления

        # 1. Обучим D: max log(D(x)) + log(1 - D(G(z)))
        
        D.zero_grad()
        
        # a) Распакуйте данные на нужный девайс
        #    Прогоните через сеть
        #    Сгенерируйте вектор из единичек (ответы для реальных сэмплов)
        #    Посчитайте лосс, сделайте .backward()
        # b) Посэмплите из torch.randn
        #    Прогоните этот шум через генератор
        #    detach-ните (нам не нужно считать градиенты G)
        #    Прогоните через дискриминатор
        #    Сгенерите вектор из нулей (ответы для фейков)
        #    Посчитайте лосс, сделайте backward (он сложится, а не перезапишется)
        #
        #    Также можно сначала сгенерировать данные, а потом собрать из двух частей батч,
        #    В котором первая половина лэйблов будет нулями, а вторая -- единицами
        
        optim_D.step()
        

        # 2. Обучим G: max log(D(G(z)))

        G.zero_grad()
        
        # Тут проще:
        #    Получим вектор неправильных ответов -- вектор единиц (мы ведь хотим, чтобы D считал их неправильными)
        #    Прогоним ранее сгенерированные картинки через D
        #    Посчитаем лосc, сделаем .backward()
        
        optim_G.step()

        # Раз в сколько-то итераций логгируем лосс
        if iters % 10 == 0:
            # Выведем информацию о том, как наша сеть справляется
            print(f'{epoch}/{num_epochs}, {iters/len(loader)}')
            print(f'  G loss: {G_loss}')
            print(f'  D loss: {D_loss}')
            print()
            
        D_losses.append(D_loss.item())
        G_losses.append(G_loss.item())

        if iters % 50 == 0:
            # вы на этом батче уже генерировали какие-то картинки: просто добавьте их в список
            
            # а вот тут сохраняем
            torch.save(D.state_dict(), '/content/drive/My Drive/D.pt')
            torch.save(G.state_dict(), '/content/drive/My Drive/G.pt')
        iters += 1

In [0]:
plt.figure(figsize=(10,5))
plt.plot(G_losses, label="G")
plt.plot(D_losses, label="D")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [0]:
# распечатайте ваши картинки

### Что дальше?

Довольно старый, но актуальный список трюков: https://github.com/soumith/ganhacks

Вообще, теория сходимости GANов очень сильно развилась за последнее время. Если хотите во всём этом разобраться, то возьмите какую-нибудь [достаточно новую статью](https://arxiv.org/pdf/1802.05957.pdf) и рекурсивно почитайте оттуда абстракты из списока литературы.